In [5]:
import serial
import time

# Configurar el puerto serial
ser = serial.Serial('COM3', 9600, timeout=1)

def send_data(data):
    try:
        ser.write(data.encode())
        print(f"Sent: {data}")
    except Exception as e:
        print(f"Error sending data: {e}")

def main():
    try:
        if not ser.is_open:
            ser.open()

        # Enviar datos periódicamente
        while True:
            data = "Hello, Arduino!"
            send_data(data)
            time.sleep(2)  # Esperar 2 segundos antes de enviar el próximo dato

    except KeyboardInterrupt:
        print("Program interrupted")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        if ser.is_open:
            ser.close()

if __name__ == "__main__":
    main()


Sent: Hello, Arduino!
Sent: Hello, Arduino!
Sent: Hello, Arduino!
Sent: Hello, Arduino!
Sent: Hello, Arduino!
Program interrupted


In [15]:
import requests

url = "https://rockblock.rock7.com/rockblock/MT?imei=300434064043170&username=jesus%40jqbit.com&password=828house&data=AABBAAF3&flush=yes"
url = "https://rockblock.rock7.com/rockblock/MT?imei=300434064043170&username=jesus%40jqbit.com&password=828house&data=2043240102&flush=yes"
# url = "https://rockblock.rock7.com/rockblock/MT?imei=300434064043170&username=jesus%40jqbit.com&password=828house&data=2043240102"

headers = {"accept": "text/plain"}
print("Url : ", url)

response = requests.post(url, headers=headers)

print(response.text)

Url :  https://rockblock.rock7.com/rockblock/MT?imei=300434064043170&username=jesus%40jqbit.com&password=828house&data=2043240102&flush=yes
OK,4155381


## IC-LISTEN STREAMING

In [ ]:
#!/usr/bin/python3
import socket
import threading

# Definición de constantes
# Definición de constantes
class Constants:
    IP_ADDRESS = "192.168.10.150"
    PORT_WAVEFORM_UDP = 51676
    PORT_SPECTRUM_UDP = 51677
    PORT_WAVEFORM_TCP = 51678
    PORT_SPECTRUM_TCP = 51679    
    START_STREAM_COMMAND = bytes([0x33, 0x2A, 0x00, 0x04, 0x00, 0x01, 0x00, 0x00])
    SOCKET_TIMEOUT = 5

# Función para manejar la recepción de datos y guardar en archivo
def handle_socket(sock, filename):
    with open(filename, 'wb') as f:
        while True:
            data = sock.recv(1024)
            print("File: ", filename, end="")
            print("Data: ", ' '.join(format(x, '02x') for x in data[:10]))
            # Pirnt the data as bytes
            if not data:
                break
            f.write(data)

def main():
    # Crear sockets y enviar el comando de inicio de flujo
    waveform_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    spectrum_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    waveform_socket.settimeout(Constants.SOCKET_TIMEOUT)
    spectrum_socket.settimeout(Constants.SOCKET_TIMEOUT)

    try:
        waveform_socket.connect((Constants.IP_ADDRESS, Constants.PORT_WAVEFORM_TCP))
        waveform_socket.sendall(Constants.START_STREAM_COMMAND)
        
        spectrum_socket.connect((Constants.IP_ADDRESS, Constants.PORT_SPECTRUM_TCP))
        spectrum_socket.sendall(Constants.START_STREAM_COMMAND)
        
        # Crear hilos para escuchar y guardar los datos recibidos
        waveform_thread = threading.Thread(target=handle_socket, args=(waveform_socket, 'data.wav'))
        spectrum_thread = threading.Thread(target=handle_socket, args=(spectrum_socket, 'data.fft'))
        
        # Iniciar los hilos
        waveform_thread.start()
        spectrum_thread.start()
        
        # Esperar a que los hilos terminen (lo cual en este caso no ocurrirá ya que son infinitos)
        waveform_thread.join()
        spectrum_thread.join()

    finally:
        waveform_socket.close()
        spectrum_socket.close()

if __name__ == "__main__":
    main()


# STOP ALL STREAMS

In [ ]:
#!/usr/bin/python3
import socket
import threading

# Definición de constantes
class Constants:
    IP_ADDRESS = "192.168.10.150"
    PORT_WAVEFORM_UDP = 51676
    PORT_SPECTRUM_UDP = 51677
    PORT_WAVEFORM_TCP = 51678
    PORT_SPECTRUM_TCP = 51679
    PORT_COMMAND_TCP = 50000
    START_STREAM_COMMAND = bytes([0x33, 0x2A, 0x00, 0x04, 0x00, 0x01, 0x00, 0x00])    
    SOCKET_TIMEOUT = 5

# CRC-16 lookup table
crc16_table = [
    0x0000, 0xc0c1, 0xc181, 0x0140, 0xc301, 0x03c0, 0x0280, 0xc241,
    0xc601, 0x06c0, 0x0780, 0xc741, 0x0500, 0xc5c1, 0xc481, 0x0440,
    0xcc01, 0x0cc0, 0x0d80, 0xcd41, 0x0f00, 0xcfc1, 0xce81, 0x0e40,
    0x0a00, 0xcac1, 0xcb81, 0x0b40, 0xc901, 0x09c0, 0x0880, 0xc841,
    0xd801, 0x18c0, 0x1980, 0xd941, 0x1b00, 0xdbc1, 0xda81, 0x1a40,
    0x1e00, 0xdec1, 0xdf81, 0x1f40, 0xdd01, 0x1dc0, 0x1c80, 0xdc41,
    0x1400, 0xd4c1, 0xd581, 0x1540, 0xd701, 0x17c0, 0x1680, 0xd641,
    0xd201, 0x12c0, 0x1380, 0xd341, 0x1100, 0xd1c1, 0xd081, 0x1040,
    0xf001, 0x30c0, 0x3180, 0xf141, 0x3300, 0xf3c1, 0xf281, 0x3240,
    0x3600, 0xf6c1, 0xf781, 0x3740, 0xf501, 0x35c0, 0x3480, 0xf441,
    0x3c00, 0xfcc1, 0xfd81, 0x3d40, 0xff01, 0x3fc0, 0x3e80, 0xfe41,
    0xfa01, 0x3ac0, 0x3b80, 0xfb41, 0x3900, 0xf9c1, 0xf881, 0x3840,
    0x2800, 0xe8c1, 0xe981, 0x2940, 0xeb01, 0x2bc0, 0x2a80, 0xea41,
    0xee01, 0x2ec0, 0x2f80, 0xef41, 0x2d00, 0xedc1, 0xec81, 0x2c40,
    0xe401, 0x24c0, 0x2580, 0xe541, 0x2700, 0xe7c1, 0xe681, 0x2640,
    0x2200, 0xe2c1, 0xe381, 0x2340, 0xe101, 0x21c0, 0x2080, 0xe041,
    0xa001, 0x60c0, 0x6180, 0xa141, 0x6300, 0xa3c1, 0xa281, 0x6240,
    0x6600, 0xa6c1, 0xa781, 0x6740, 0xa501, 0x65c0, 0x6480, 0xa441,
    0x6c00, 0xacc1, 0xad81, 0x6d40, 0xaf01, 0x6fc0, 0x6e80, 0xae41,
    0xaa01, 0x6ac0, 0x6b80, 0xab41, 0x6900, 0xa9c1, 0xa881, 0x6840,
    0x7800, 0xb8c1, 0xb981, 0x7940, 0xbb01, 0x7bc0, 0x7a80, 0xba41,
    0xbe01, 0x7ec0, 0x7f80, 0xbf41, 0x7d00, 0xbdc1, 0xbc81, 0x7c40,
    0xb401, 0x74c0, 0x7580, 0xb541, 0x7700, 0xb7c1, 0xb681, 0x7640,
    0x7200, 0xb2c1, 0xb381, 0x7340, 0xb101, 0x71c0, 0x7080, 0xb041,
    0x5000, 0x90c1, 0x9181, 0x5140, 0x9301, 0x53c0, 0x5280, 0x9241,
    0x9601, 0x56c0, 0x5780, 0x9741, 0x5500, 0x95c1, 0x9481, 0x5440,
    0x9c01, 0x5cc0, 0x5d80, 0x9d41, 0x5f00, 0x9fc1, 0x9e81, 0x5e40,
    0x5a00, 0x9ac1, 0x9b81, 0x5b40, 0x9901, 0x59c0, 0x5880, 0x9841,
    0x8801, 0x48c0, 0x4980, 0x8941, 0x4b00, 0x8bc1, 0x8a81, 0x4a40,
    0x4e00, 0x8ec1, 0x8f81, 0x4f40, 0x8d01, 0x4dc0, 0x4c80, 0x8c41,
    0x4400, 0x84c1, 0x8581, 0x4540, 0x8701, 0x47c0, 0x4680, 0x8641,
    0x8201, 0x42c0, 0x4380, 0x8341, 0x4100, 0x81c1, 0x8081, 0x4040,
]

# Función para calcular el CRC-16
def calculate_crc(data):
    checksum = 0
    for byte in data:
        checksum = crc_update(byte, checksum)
    return checksum

def crc_update(data, accum):
    comb_value = (accum & 0xff) ^ data
    accum = (accum >> 8) ^ crc16_table[comb_value]
    return accum


def main():
    # Crear sockets y enviar el comando de inicio de flujo
    command_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    # Configure the socket timeout
    command_socket.settimeout(Constants.SOCKET_TIMEOUT)
    
    try:
        # Calcular el CRC y preparar el comando (STOP ALL STREAMS)
        command = [0x2A, 0x51, 0x00, 0x00]
        crc = calculate_crc(command)
        command.append(crc & 0xFF)
        command.append(crc >> 8)
        command = bytes(command)
        
        print("Command: ", ' '.join(format(x, '02x') for x in command))

        command_socket.connect((Constants.IP_ADDRESS, Constants.PORT_COMMAND_TCP))
        command_socket.sendall(command)
        
        # Receive the response
        response = command_socket.recv(1024)
        print("Response: ", ' '.join(format(x, '02x') for x in response))
                
    finally:
        command_socket.close()

if __name__ == "__main__":
    main()


# Test Raspberry PI3 API

In [15]:
import struct

# Cabecera
sync_byte = 0x20
command_code = ord('R')  # Usando 'R' como ejemplo, cambia según el comando correcto
addresses = (0x02 << 6 | 0x03 << 4 | 0x04)  # Ejemplo de combinación de direcciones

# Payload
recording_wav = 0
recording_fft = 0
processing_wav = 1
processing_fft = 0
wav_processing_type = 2
fft_processing_type = 3
wav_period = 300  # Ejemplo de periodo WAV
wav_duration = 60  # Ejemplo de duración WAV
fft_period = 600  # Ejemplo de periodo FFT
fft_duration = 120  # Ejemplo de duración FFT

# Byte 0
byte0 = (recording_wav << 3) | (recording_fft << 2) | (processing_wav << 1) | processing_fft
print("Byte 0: ", byte0)

# Byte 1
byte1 = (wav_processing_type << 4) | fft_processing_type

# Construir el payload
payload = bytearray([byte0, byte1])
payload.extend(struct.pack('>H', wav_period))  # '>H' para big-endian uint16_t
payload.extend(struct.pack('>H', wav_duration))
payload.extend(struct.pack('>H', fft_period))
payload.extend(struct.pack('>H', fft_duration))

# Longitud del payload
payload_length = len(payload)

# Vector de bytes completo
byte_vector = bytearray([sync_byte, command_code, addresses, payload_length])
byte_vector.extend(payload)

# Imprimir el vector de bytes
print(
    ' '.join(format(x, '02x') for x in byte_vector),
)


Byte 0:  2
20 52 b4 0a 02 23 01 2c 00 3c 02 58 00 78


In [22]:
import serial
import time


# Configurar el puerto serial
ser = serial.Serial('COM3', 9600, timeout=1)

def send_data(byte_vector):
    try:
        ser.write(byte_vector)
        print(f"Sent: {byte_vector}")
    except Exception as e:
        print(f"Error sending data: {e}")

def main():
    try:
        if not ser.is_open:
            ser.open()


        get_summary = bytearray([0x20, ord('S'), (0x02 << 6 | 0x03 << 4 | 0x04 ), 0x00])  # Ejemplo de vector de bytes
        # Sync byte (0x20), CommandCode (0x53), Addresses (0x23), PayloadLength (0x00)
        get_rec_config = bytearray([0x20, ord('r'), (0x02 << 6 | 0x03 << 4 | 0x04 ), 0x00])  # Ejemplo de vector de bytes
        #20 52 b4 0a 02 23 01 2c 00 3c 02 58 00 78
        set_rec_config = bytearray([0x20, ord('R'), (0x02 << 6 | 0x03 << 4 | 0x04 ), 
                                    0x0A, 0x02, 0x23, 0x01, 0x2C, 0x00, 0x3C, 0x02, 0x58, 0x00, 0x78
                                    ])  
        #20 52 b4 0a 00 00 00 01 00 01 00 00 00 00
        reset_rec_config = bytearray([0x20, ord('R'), (0x02 << 6 | 0x03 << 4 | 0x04 ),
                                    0x0A, 0x00, 0x00, 0x00, 0x01, 0x00, 0x01, 0x00, 0x00, 0x00, 0x00
                                    ])                                                                          
        send_data(reset_rec_config)
        # REQ : 10110100 -> RES: 11100100

    except KeyboardInterrupt:
        print("Program interrupted")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        if ser.is_open:
            ser.close()

if __name__ == "__main__":
    main()


Sent: bytearray(b' R\xb4\n\x00\x00\x00\x01\x00\x01\x00\x00\x00\x00')
